# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

In [1]:
import pandas as pd
import numpy as np
import math
import json
import warnings
warnings.filterwarnings('ignore')

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In order to properly analyze the data, some rangling needs to take place

In [2]:
# replace space with _ in values in event column
transcript['event'] = transcript['event'].str.replace(' ', '_')

# change person column name to id
transcript.rename(columns={'person': 'id'}, inplace=True)

In [3]:
def data_cleaner(portfolio, profile, transcript, event):
        """
        This function takes in data from three sources,
        including the customer profile, the portfolio of
        offers, and the transaction history. It cleans the
        data and then merges the data into a high quality 
        dataset suitable for analysis or machine learning.
        
        Args:
        portfolio: the portfolio df
        profile: the profile df
        transcript: the transcript df
        type: the type of event to filter out

        Returns:
        offer_completed: the cleaned offer_completed df

        """
        # filter out offer completed events
        if event == 'offer_completed':
                transactions = transcript.loc[transcript['event']== 'offer_completed']     
        elif event == 'offer_received':    
                transactions = transcript.loc[transcript['event']== 'offer_received']
        elif event == 'offer_viewed':
                transactions = transcript.loc[transcript['event']== 'offer_viewed']
        else:
                transactions = transcript.loc[transcript['event']== 'transaction']

        # extract offer_id and reward from value column
        offer_id = pd.json_normalize(transactions['value'])   
        # update index to match offer_completed for merge
        offer_id.index = transactions.index
        # merge offer_completed with offer_id_reward
        transactions = pd.concat([transactions, offer_id], axis=1)
        # drop value column
        transactions.drop(columns=['value'], inplace=True)

        if event == 'transaction':
                return transactions

        # rename offer_id column
        transactions.rename(columns={'offer id': 'offer_id'}, inplace=True)

        # change id column in portfolio to offer_id
        portfolio.rename(columns={'id': 'offer_id'}, inplace=True)
        

        # set condition where completed offer has reward column
        if event == 'offer_completed':
                # merge offer_completed with portfolio
                transactions = pd.merge(transactions, portfolio, on=['reward','offer_id'], how='left')
        else: # all other conditions
                transactions = pd.merge(transactions, portfolio, on=['offer_id'], how='left')
        # remove the [] from channels column
        transactions['channels'] = transactions['channels'].apply(lambda x: ','.join(map(str, x)))
        # create dummy variables for channels column, join to main df
        transactions = transactions.join(transactions['channels'].str.get_dummies(','))
        # drop unnecessary columns
        transactions.drop(columns='channels', inplace=True)
        # drop offer_id column
        transactions.drop(columns='offer_id', inplace=True)
        

        return transactions

In [4]:
# clean the dataframes

transactions = data_cleaner(portfolio, profile, transcript, 'transaction')
offer_completed = data_cleaner(portfolio, profile, transcript, 'offer_completed')
offer_viewed = data_cleaner(portfolio, profile, transcript, 'offer_viewed')
offer_received = data_cleaner(portfolio, profile, transcript, 'offer_received')


In [5]:
# view transactions
transactions

,id,event,time,amount
12654,02c083884c7d45b39cc68e1314fec56c,transaction,0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,34.56
12659,54890f68699049c2a04d415abc25e717,transaction,0,13.23
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,19.51
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,18.97
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53


In [6]:
# view the offers completed dataset
offer_completed

,id,event,time,reward,difficulty,duration,offer_type,email,mobile,social,web
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer_completed,0,2,10,7,discount,1,1,0,1
1,fe97aa22dd3e48c8b143116a8403dd52,offer_completed,0,2,10,10,discount,1,1,1,1
2,629fc02d56414d91bca360decdfa9288,offer_completed,0,5,5,7,bogo,1,1,0,1
3,676506bad68e4161b9bbaffeb039626b,offer_completed,0,10,10,7,bogo,1,1,1,0
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer_completed,0,10,10,5,bogo,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
33574,0c027f5f34dd4b9eba0a25785c611273,offer_completed,714,3,7,7,discount,1,1,1,1
33575,a6f84f4e976f44508c358cc9aba6d2b3,offer_completed,714,3,7,7,discount,1,1,1,1
33576,b895c57e8cd047a8872ce02aa54759d6,offer_completed,714,2,10,10,discount,1,1,1,1
33577,8431c16f8e1d440880db371a68f82dd0,offer_completed,714,2,10,10,discount,1,1,1,1


In [7]:
# merge the dfs

from functools import reduce

dfs = [offer_completed, offer_viewed, offer_received]

# merge the dataframes
offers = (
        reduce(lambda left, right: pd.merge(left, right, 
        on=['id', 'event', 'time', 'reward', 'difficulty', 'duration', 
        'offer_type', 'email', 'mobile', 'social', 'web'], 
        how='outer'), dfs)
        )

offers

,id,event,time,reward,difficulty,duration,offer_type,email,mobile,social,web
0,0009655768c64bdeb2e877511632db8f,offer_completed,414,5,5,5,bogo,1,1,1,1
1,0009655768c64bdeb2e877511632db8f,offer_completed,528,2,10,10,discount,1,1,1,1
2,0009655768c64bdeb2e877511632db8f,offer_completed,576,2,10,7,discount,1,1,0,1
3,0009655768c64bdeb2e877511632db8f,offer_received,168,0,0,3,informational,1,1,1,0
4,0009655768c64bdeb2e877511632db8f,offer_received,336,0,0,4,informational,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
167576,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,174,5,20,10,discount,1,0,0,1
167577,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,354,2,10,7,discount,1,1,0,1
167578,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,414,2,10,7,discount,1,1,0,1
167579,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,534,5,5,7,bogo,1,1,0,1


In [8]:
# transactions that do not appear in offers
not_in_offers = transactions.loc[~transactions['id'].isin(offers['id'])]

not_in_offers
                

,id,event,time,amount
16379,eb540099db834cf59001f83a4561aef3,transaction,6,4.74
22092,12ede229379747bd8d74ccdc20097ca3,transaction,18,16.62
69477,c6e579c6821c41d1a7a6a9cf936e91bb,transaction,174,0.65
87239,c6e579c6821c41d1a7a6a9cf936e91bb,transaction,222,1.91
106812,ae8111e7e8cd4b60a8d35c42c1110555,transaction,306,17.51
108268,eb540099db834cf59001f83a4561aef3,transaction,318,5.09
109340,3a4874d8f0ef42b9a1b72294902afea9,transaction,324,22.75
109815,12ede229379747bd8d74ccdc20097ca3,transaction,324,27.60
110707,ae8111e7e8cd4b60a8d35c42c1110555,transaction,330,28.70
142347,3a4874d8f0ef42b9a1b72294902afea9,transaction,378,27.20


In [9]:
offers

,id,event,time,reward,difficulty,duration,offer_type,email,mobile,social,web
0,0009655768c64bdeb2e877511632db8f,offer_completed,414,5,5,5,bogo,1,1,1,1
1,0009655768c64bdeb2e877511632db8f,offer_completed,528,2,10,10,discount,1,1,1,1
2,0009655768c64bdeb2e877511632db8f,offer_completed,576,2,10,7,discount,1,1,0,1
3,0009655768c64bdeb2e877511632db8f,offer_received,168,0,0,3,informational,1,1,1,0
4,0009655768c64bdeb2e877511632db8f,offer_received,336,0,0,4,informational,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
167576,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,174,5,20,10,discount,1,0,0,1
167577,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,354,2,10,7,discount,1,1,0,1
167578,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,414,2,10,7,discount,1,1,0,1
167579,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,534,5,5,7,bogo,1,1,0,1


In [10]:
offers.shape

(167581, 11)

In [11]:
# create a one:many relationship between offers and profile
# for every offer, we will merge the profile data

# merge offers with profile
offers = pd.merge(offers, profile, on='id', how='left')

offers

,id,event,time,reward,difficulty,duration,offer_type,email,mobile,social,web,gender,age,became_member_on,income
0,0009655768c64bdeb2e877511632db8f,offer_completed,414,5,5,5,bogo,1,1,1,1,M,33,20170421,72000.0
1,0009655768c64bdeb2e877511632db8f,offer_completed,528,2,10,10,discount,1,1,1,1,M,33,20170421,72000.0
2,0009655768c64bdeb2e877511632db8f,offer_completed,576,2,10,7,discount,1,1,0,1,M,33,20170421,72000.0
3,0009655768c64bdeb2e877511632db8f,offer_received,168,0,0,3,informational,1,1,1,0,M,33,20170421,72000.0
4,0009655768c64bdeb2e877511632db8f,offer_received,336,0,0,4,informational,1,1,0,1,M,33,20170421,72000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167576,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,174,5,20,10,discount,1,0,0,1,F,45,20161125,62000.0
167577,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,354,2,10,7,discount,1,1,0,1,F,45,20161125,62000.0
167578,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,414,2,10,7,discount,1,1,0,1,F,45,20161125,62000.0
167579,ffff82501cea40309d5fdd7edcca4a07,offer_viewed,534,5,5,7,bogo,1,1,0,1,F,45,20161125,62000.0


In [12]:
# create crosstab to evaluate offer type and event
cross_tab = (
            pd.pivot_table(offers, 
                           index='offer_type', 
                           columns='event', 
                           values='id', 
                           aggfunc='count', 
                           fill_value=0)
            .reset_index()
            )

# sort by offer_completed
cross_tab = cross_tab.sort_values(by='offer_completed', ascending=False)

# calculate conversion rate from received to completed
cross_tab['viewed_rate'] = cross_tab['offer_completed'] / cross_tab['offer_received']

# calculate conversion rate from viewed to completed
cross_tab['completed_rate'] = cross_tab['offer_completed'] / cross_tab['offer_viewed']

cross_tab

event,offer_type,offer_completed,offer_received,offer_viewed,viewed_rate,completed_rate
1,discount,17910,30543,21445,0.586386,0.835160
0,bogo,15669,30499,25449,0.513755,0.615702
2,informational,0,15235,10831,0.000000,0.000000


In [13]:
# find orders that were completed but offer was not viewed
not_viewed = transaction_update.loc[
                                    (transaction_update['offer_completed'] == 1) &
                                    (transaction_update['offer_viewed'] == 0) &
                                    (transaction_update['offer_received'] == 1)
                                    ]
# populate order values
not_viewed.loc[:,'amount'] = (
                                not_viewed
                                .loc[:,'id']
                                .apply(lambda x: transactions.loc[transactions['id'] == x]['amount']
                                .sum())
                            )
# add demographic data
not_viewed = pd.merge(not_viewed, profile, on='id', how='left')

# preview first few rows
not_viewed

NameError: name 'transaction_update' is not defined

In [ ]:
# get sum of all transactions by id
sum_by_id = transactions.groupby('id')['amount'].sum().reset_index()

# add amount colum with one:many merge
offers = pd.merge(offers, sum_by_id, on='id', how='left')

offers

In [ ]:
# customer, offer, transaction data snapshot

# set samples
n = 10

# create a snapshot of the data
customer_snapshot = (
                    offers
                    .sample(n)
                    [['event', 'offer_type','gender','age','income','amount','became_member_on']]
                    .sort_values(by='amount', ascending=False)
                    )

customer_snapshot

In [ ]:
# evaluate account growth by membership date
# convert to datetime
offers['became_member_on'] = pd.to_datetime(offers['became_member_on'], format='%Y%m%d')

# create a new column for year
offers['year'] = offers['became_member_on'].dt.year


In [ ]:
membership_growth = (
                    pd.pivot_table(offers, 
                                   columns='year', 
                                   values='id', 
                                   aggfunc='count', 
                                   fill_value=0)
                                   .sum()
                                   .to_frame()
                                   .reset_index()
                                   .rename(columns={'index': 'year', 0: 'count'})
                    )

# calculate growth rate
membership_growth['growth_rate'] = membership_growth['count'].pct_change().fillna(0)

# preview
membership_growth



### Visualize the Baseline Data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Convert 'year' and 'count' columns to numpy arrays
membership_growth['year'] = np.array(membership_growth['year'])
membership_growth['count'] = np.array(membership_growth['count'])

# bar plot
fig, ax = plt.subplots(figsize=(10, 6))

# Add trendline using regplot
sns.barplot(x='year', y='count', data=membership_growth, ax=ax)

plt.title('Membership Growth by Year')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()

In [ ]:
# factorize id column
offers['id'] = pd.factorize(offers['id'])[0]

offers


### Most Offers by Duration

In [ ]:
# plot the distribution of offer duration
offers.groupby(['duration'])['reward'].count().to_frame().plot(kind='bar')
plt.xticks(rotation=0)
# remove legend
plt.legend().remove()

In [ ]:
import pandas as pd

# Group by 'duration' and 'event', and count the number of 'amount'
counts = offers.groupby(['duration', 'event'])['amount'].count().reset_index()

# Rename the 'amount' column to 'count'
counts.rename(columns={'amount': 'count'}, inplace=True)

# Now plot the data
sns.barplot(x='duration', y='count', hue='event', data=counts)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.title('Count by Duration and Event')

In [ ]:
# sum of purchases by year
offers.groupby(['id','year'])['amount'].sum().reset_index().plot(x='year', y='amount', kind='scatter')
plt.title('Total Purchases by Year')
plt.xlabel('Year')
plt.ylabel('Amount Purchased')

In [ ]:
# time series
offers['day'] = pd.to_datetime(offers['became_member_on'], unit='d').dt.day
offers['day_of_year'] = pd.to_datetime(offers['became_member_on'], unit='d').dt.dayofyear
offers['month'] = pd.to_datetime(offers['became_member_on'], unit='d').dt.month
offers['week'] = pd.to_datetime(offers['became_member_on'], unit='d').dt.isocalendar().week


In [ ]:
# create a separate time series plot for each year from 2013 - 2018
fig = plt.figure(figsize=(15, 15))
for i, year in enumerate(range(2013, 2019)):
    ax = plt.subplot(3, 2, i + 1)
    offers.loc[offers['year'] == year].groupby('day_of_year')['amount'].sum().plot(ax=ax)
    plt.title(f'Total Purchases by day of Year in $\mathbf{{{year}}}$', fontsize=14)
    plt.xlabel('Day of Year')
    plt.ylabel('Amount Purchased')
    # create more space between plot height and width
    plt.subplots_adjust(hspace=0.5, wspace=0.5)

### Review Order Types by Demographic Groups

In [ ]:
offers.columns

In [ ]:
demo = offers[['event','duration','offer_type','gender','age','income','became_member_on','amount']]

demo

In [ ]:
demo.agg(
        {'age': ['min', 'max', 'mean', 'median'],'gender': ['count']}
        ).fillna(0)

In [ ]:
demo_amount = (
                demo.groupby(['age'])['amount']
                .sum()
                .reset_index()
                .sort_values(by='amount', ascending=False)
              )

# percent of total amount
demo_amount['percent_total'] = (demo_amount['amount'] / demo_amount['amount'].sum()).round(4)

# total amount $19.7m
demo_amount.amount.sum()

demo_amount